<a href="https://colab.research.google.com/github/audreyemmely/pln/blob/main/pln_lista4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Resolução Lista 4**

---

Audrey Emmely Rodrigues Vasconcelos

Karen Nayara Gomes da Silva

In [1]:
# from google.colab import files
# uploaded = files.upload()
# upando train_data e test_data_solution

In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF
import warnings
warnings.filterwarnings('ignore')

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
df = pd.read_csv('train_data.txt', sep =':::', header = None, engine = 'python', nrows = 10000)
df.columns = ['id', 'title', 'genre', 'description']
df2 = pd.read_csv('test_data_solution.txt', sep =':::', header = None, engine = 'python', nrows = 10000)
df2.columns = ['id', 'title', 'genre', 'description']

In [5]:
dataset = pd.concat([df, df2], ignore_index=True)
dataset.drop(['id','genre'], axis=1, inplace=True)

In [6]:
def preprocess(description):
  description = re.sub(r'\w*\d\w*', '', description) #remove todas as palavras que contêm números
  description = re.sub(r'[^a-zA-Z ]', '', description.lower())
  return re.sub(r'\s+', ' ', description) #retira espaços repetidos

In [7]:
dataset['processed_description'] = dataset.description.apply(preprocess)

In [8]:
stop_words = set(nltk.corpus.stopwords.words('english'))
def remove_stopwords(description):
    tokenized_text = nltk.word_tokenize(description, language='english')
    return " ".join([token for token in tokenized_text if token not in stop_words])

In [9]:
dataset['processed_description'] = dataset.processed_description.apply(remove_stopwords)

In [10]:
dataset

,title,description,processed_description
0,Oscar et la dame rose (2009),Listening in to a conversation between his do...,listening conversation doctor parents yearold ...
1,Cupid (1997),A brother and sister with a past incestuous r...,brother sister past incestuous relationship cu...
2,"Young, Wild and Wonderful (1980)",As the bus empties the students for their fie...,bus empties students field trip museum natural...
3,The Secret Sin (1915),To help their unemployed father make ends mee...,help unemployed father make ends meet edith tw...
4,The Unrecovered (2007),The film's title refers not only to the un-re...,films title refers unrecovered bodies ground z...
...,...,...,...
19995,9/11: Millionaire Widows (2008),"In the wake of the events of 9/11, nearly 300...",wake events nearly families slated receive nea...
19996,Gangsters of the Frontier (1944),Tex put the Kern gang away once but they have...,tex put kern gang away returned reinforcements...
19997,Saheb Bahadur (1977),"In a small town, situated in a picturesque In...",small town situated picturesque indian valley ...
19998,Sitting in Limbo (1997),"Easter Sunday, 1955. A Sunday school classroo...",easter sunday sunday school classroom argument...


## Questão 1

Realize um comparativo entre os métodos LDA, SVD e NMF para realizar
modelagem de tópicos. Para cada método:

a) empiricamente, tente identificar uma quantidade de tópicos adequada para
seu problema. Exiba resultados que justifiquem a quantidade de tópicos
escolhida;

b) identifique as 5 palavras mais relevantes de cada tópico;

c) identifique os 3 tópicos mais relevantes de 5 documentos quaisquer (você
pode representar os tópicos por suas 5 palavras mais relevantes).

d) qual método apresentou melhores resultados, na sua opinião? Justifique com resultados/dados.

In [11]:
n_topics = 5
samples = dataset.sample(n=5)
samples.reset_index(inplace=True)
samples

,index,title,description,processed_description
0,4410,In the Face of Evil: Reagan's War in Word and...,Ronald Reagan was the President of the Screen...,ronald reagan president screen actors guild in...
1,7107,The Game (2017/I),The Game continues where The Secret left off....,game continues secret left empowering intimate...
2,6296,Ezra Crane (2008),Wars raged. Plagues ravaged. Civilization fel...,wars raged plagues ravaged civilization fell n...
3,17323,Face of Fire (1959),"In 1898, in a small American town, Dr. Ned Tr...",small american town dr ned trescott family liv...
4,13522,No Outlet (2007),Middle-class America supports a rich irony: i...,middleclass america supports rich irony childr...


Utilizando TF-IDF como embbeding dos textos

In [12]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(dataset.processed_description)

In [13]:
for decomposition_algorithm in (LatentDirichletAllocation, TruncatedSVD, NMF):
    algorithm = decomposition_algorithm(n_components=n_topics)
    X_topics = algorithm.fit_transform(X)

    def get_top_words(topic, n, words, algorithm):
      sorted_idxs = np.argsort(algorithm.components_[topic])
      top_words = [words[i] for i in sorted_idxs]
      return [w for w in reversed(top_words[-n:])]

    print(f'Algoritmo: {decomposition_algorithm.__name__}')
    for i in range(n_topics):
        print(f'Tópico {i} -> {get_top_words(i, 5, tfidf.get_feature_names(), algorithm)}')
    print('\n')

    sample_topics = algorithm.transform(tfidf.transform(samples.processed_description))

    for i, sample in samples.iterrows():
        print(f'{sample.title}')
        print(sample.description[:70] + '...')
        print(sample_topics[i])
        print('\n')

Algoritmo: LatentDirichletAllocation
Tópico 0 -> ['nash', 'lila', 'maud', 'minerva', 'offroad']
Tópico 1 -> ['di', 'everest', 'liza', 'doyle', 'ephesus']
Tópico 2 -> ['life', 'one', 'film', 'new', 'young']
Tópico 3 -> ['de', 'minas', 'gerais', 'brisbane', 'nana']
Tópico 4 -> ['de', 'la', 'et', 'le', 'un']


 In the Face of Evil: Reagan's War in Word and Deed (2004) 
 Ronald Reagan was the President of the Screen Actor's Guild during th...
[0.02425239 0.02425115 0.90299232 0.02425104 0.0242531 ]


 The Game (2017/I) 
 The Game continues where The Secret left off. It's an empowering, int...
[0.02464261 0.02464308 0.90142972 0.02464257 0.02464202]


 Ezra Crane (2008) 
 Wars raged. Plagues ravaged. Civilization fell. A new dark age took a...
[0.02423436 0.02423434 0.90306272 0.02423432 0.02423426]


 Face of Fire (1959) 
 In 1898, in a small American town, Dr. Ned Trescott and his family li...
[0.0229991  0.02299945 0.90800375 0.02299881 0.02299889]


 No Outlet (2007) 
 Middle-class Amer

Melhor algoritmo: NMF


## Questão 2
Realize um agrupamento dos dados seguindo os seguintes passos:

a) Aplique o algoritmo PCA, preservando 95% da variância nos dados. Qual a
dimensão resultante dos dados projetados?

b) Aplique um algoritmo k-means nos dados projetados, usando o método elbow
para encontrar o valor de k ótimo.

c) Finalmente, aplique um algoritmo k-means nos dados originais (sem projeção
por PCA), usando o método elbow para encontrar o valor de k ótimo.

**Referências**

